## 1. 파이썬과 MySQL 활용

+ 파이썬과 MySQL을 연결해주는 모듈 : mysqlclient, PyMySQL
    - pip install mysqlclient
    - pip install PyMySQL

In [1]:
! pip install mysqlclient

In [2]:
! pip install PyMySQL

In [1]:
# mysqlclient 모듈 사용
import MySQLdb

conn = MySQLdb.connect(host="127.0.0.1", user="root", password="1111", database="world")
print(conn)
conn.close()

<_mysql.connection open to '127.0.0.1' at 0000021C2BA60AD0>


In [6]:
# pymysql 모듈 사용
import pymysql

conn = pymysql.connect(host="127.0.0.1", user="root", password="1111", database="world")
print(conn)
conn.close()

In [3]:
config = {"host":"127.0.0.1", "user":"root", "password":"1111", "database":"world"}

In [13]:
conn = MySQLdb.connect(**config)
print(conn)
conn.close()

<_mysql.connection open to '127.0.0.1' at 00000275C1736F20>


#### (1) SELECT

In [18]:
conn = MySQLdb.connect(**config)

sql = "select * from city limit 10"

cur = conn.cursor()
cur.execute(sql)

# print(cur)
# for data in cur:
    # print(data)
    # print("%d %s %s %s %d"% data)
    
#for a, b, c, d, e in cur:
#    print(a, b, c, d, e)

result = list()
for data in cur:
    result.append(data)
    
conn.close()

print(result)

[(1, 'Kabul', 'AFG', 'Kabol', 1780000), (2, 'Qandahar', 'AFG', 'Qandahar', 237500), (3, 'Herat', 'AFG', 'Herat', 186800), (4, 'Mazar-e-Sharif', 'AFG', 'Balkh', 127800), (5, 'Amsterdam', 'NLD', 'Noord-Holland', 731200), (6, 'Rotterdam', 'NLD', 'Zuid-Holland', 593321), (7, 'Haag', 'NLD', 'Zuid-Holland', 440900), (8, 'Utrecht', 'NLD', 'Utrecht', 234323), (9, 'Eindhoven', 'NLD', 'Noord-Brabant', 201843), (10, 'Tilburg', 'NLD', 'Noord-Brabant', 193238)]


#### (2) INSERT

In [21]:
conn = MySQLdb.connect(**config)

sql = "create table if not exists tbltest(empno  int, ename  varchar(10), sal  int,  job varchar(10))"
cur = conn.cursor()
cur.execute(sql)

sql = "insert into tbltest(empno, ename, sal) values(1, 'scott', 5000)"
cur.execute(sql)
conn.commit()

conn.close()

In [28]:
empno = 2
ename = 'miller'
sal = 1300

conn = MySQLdb.connect(**config)
cur = conn.cursor()

sql = "insert into tbltest(empno, ename, sal) values("+str(empno)+",'"+ename+"',"+str(sal)+")"
#print(sql)
cur.execute(sql)
conn.commit()

conn.close()

In [30]:
empno = 3
ename = 'tom'
sal = 2000

conn = MySQLdb.connect(**config)
cur = conn.cursor()

sql = "insert into tbltest(empno, ename, sal) values(%s, %s, %s)"
sql_data = (str(empno), ename, str(sal))

cur.execute(sql, sql_data)
conn.commit()

conn.close()

In [34]:
try :
    conn = MySQLdb.connect(**config)
    cur = conn.cursor()
    
    empno = 4
    ename = 'jerry'
    sal = 2000
    
    sql = "insert into tbltest(empno, ename, sal) values(%s, %s, %s)"
    sql_data = (str(empno), ename, str(sal))
    
    cur.execute(sql, sql_data)
    conn.commit()
except Exception as e:
    print("error : ", e)
finally:
    conn.close() 

In [35]:
# 데이터 여러 개 입력하기
conn = MySQLdb.connect(**config)
cur = conn.cursor()

for i in range(10):
    sql = "insert into tbltest(empno, ename, sal) values(5, 'jane', 10000)"
    cur.execute(sql)
    conn.commit()
    
conn.close()

#### (3) cursor 객체의 fetchall(), fetchmany(), fetchone()

+ default는 fetchall()

In [39]:
try:
    conn = MySQLdb.connect(**config)
    cur = conn.cursor()
    
    cur.execute("select * from tbltest")
    
    #for i in cur.fetchall():
    #    print(i)
    
    #for i in cur.fetchmany(5):
    #    print(i)
    
    print(cur.fetchone())

finally:
    conn.close()

(1, 'scott', 5000, None)


#### (4) DictCursor

+ 필드명을 인덱스 대신 키로 사용

In [43]:
try:
    conn = MySQLdb.connect(**config)
    cur = conn.cursor(MySQLdb.cursors.DictCursor)
    
    cur.execute("select * from tbltest")
    
    for i in cur.fetchall():
        # print(i[0], i[1], i[2])
        print(i["empno"], i["ename"], i["sal"])
    
finally:
    conn.close()

1 scott 5000
2 miller 1300
3 tom 2000
3 tom 2000
4 jerry 2000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000
5 jane 10000


#### (5) 응용 실습1 : CSV파일을 DB에 저장

In [ ]:
"""
CREATE TABLE IF NOT EXISTS suppliers(
    Supplier_Name    varchar(20),
    Invoice_Number   varchar(20),
    Part_Number      varchar(20),
    Cost             float,
    Purchase_Date    date
);

"""

In [5]:
import csv
import datetime

file = csv.reader(open("data/input.csv"))
print(file)

# print(next(file))
# print(next(file))
# print(next(file))

header = next(file)

try:
    conn = MySQLdb.connect(**config)
    cur = conn.cursor()
    
    cur.execute("delete from suppliers")
    conn.commit()
    
    for row in file:
        # print(row)
        # cur.execute("insert into suppliers values(%s, %s, %s, %s, %s)", row)
        data = []
        for col in range(len(header)):
            if col == 4:
                tmp = datetime.datetime.strptime(row[col], '%Y-%m-%d')
                data.append(tmp)
            else:
                data.append(row[col])
                
        cur.execute("insert into suppliers values(%s, %s, %s, %s, %s)", data)
        
    conn.commit()
finally:
    conn.close()

#### (5) DB에서 가져온 데이터를 파일로 저장

In [11]:
try:
    conn = MySQLdb.connect(**config)
    cur = conn.cursor()
    
    cur.execute("select * from suppliers where cost > 500000")
    
    #for i in cur:
    #    print(i)
    
    file = csv.writer(open("data/output.csv", "w"), delimiter=",")
    
    header = ["Supplier Name","Invoice Number","Part Number","Cost","Purchase Date"]
    file.writerow(header)
    
    for row in cur:
        file.writerow(row)
    
finally:
    conn.close()

## 2. 파이썬과 SQLite

1. 로컬로 사용할 수 있는 DBMS
    + Access
    + SQLite
    + Microsoft SQL Server Compact Edition
    + Firebird Embedded Server
    + H2
    + HSQL
    + Apache Derby
    + ...
    
    
2. 특징
    + 파일 기반의 DBMS, 저메모리, 빠른 처리속도
    + 오픈 소스
    + 별도의 DB서버가 없어도 쉽고 편리하게 사용할 수 있는 Embeded SQL DB엔진
    + 안드로이드, 아이폰 등의 스마트폰에 기본 내장된 DB
    + 파이썬에도 내장
    + 표준SQL 지원
    -------------------------
    + 컬럼을 삭제하거나 변경하는 것 등이 제한
    + Left Outer Join 만 지원
    + 읽기 전용 뷰만 제공
    + ...
    
    
3. 공식 사이트
    + http://sqlite.org
    
    
4. 클라이언트 툴
    + http://www.sqliteexpert.com

In [12]:
import sqlite3

In [14]:
print(sqlite3.version)
print(sqlite3.sqlite_version)

2.6.0
3.33.0


In [16]:
conn = sqlite3.connect("data/sqlitetest.db")
print(conn)
conn.close()

In [17]:
# select

conn = sqlite3.connect("data/sqlitetest.db")
cur = conn.cursor()

cur.execute("select * from tblbook")
for row in cur:
    print(row)

conn.close()

('최주현', '하늘과 땅')
('최주현', '바다')
('유은정', '바다')
('박성우', '문')
('최주현', '문')
('박성우', '천국')
('최지은', '천국')
('최주현', '천국')
('박성우', '고슴도치')
('서금동', '나')


In [ ]:
"""
CREATE TABLE tblMember(
    id   int,
    name  varchar(10),
    regdate  datetime
);

insert into tblMember(id, name, regdate) values(1, '홍길동', '2020-12-25');
insert into tblMember(id, name, regdate) values(2, '임꺽정', date('now'));
"""

In [18]:
conn = sqlite3.connect("data/sqlitetest.db")
cur = conn.cursor()

cur.execute("select * from tblMember")
for row in cur:
    print(row)

conn.close()

(1, '홍길동', '2020-12-25')
(2, '임꺽정', '2021-04-27')


In [21]:
# 자동으로 db 생성하고 테이블 생성

conn = sqlite3.connect("data/auto.db")
# print(conn)
cur = conn.cursor()

cur.execute("create table if not exists tblmember(id   int, name varchar(10))")
conn.commit()

cur.execute("insert into tblmember values(1, '홍길동')")
conn.commit()

cur.execute("select * from tblMember")
for row in cur:
    print(row)

conn.close()

(1, '홍길동')
(1, '홍길동')
